In [561]:
### Script for Pathway enrichment analysis on MOFA results; requires a prepared pathway set input dataset

#############################################
# Prerequisites - Load Libraries

In [562]:
source('MS1_Functions.r')

In [563]:
### Inform about execution start
popup_function_pos('06_Downstream_Pathways: Execution Started')

In [564]:
source('MS0_Libraries.r')

[1] "/opt/conda/envs/mofa_analysis/lib/R/library"


In [565]:
source('MS2_Plot_Config.r')

###############################################
# Preqrequisites Configurations & Parameters

In [566]:
### Load the parameters that are set via the configuration files

In [567]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'"


In [568]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/home/icb/corinna.losert/projects/mofa_workflow/input_data/
2,result_path,/home/icb/corinna.losert/projects/mofa_workflow/result_data/


In [569]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [570]:
data_path

[1] "/home/icb/corinna.losert/projects/mofa_workflow/input_data/"

In [571]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [572]:
result_path

[1] "/home/icb/corinna.losert/projects/mofa_workflow/result_data/"

In [573]:
### Load the configuration for the pathway enrichment from the config file

In [574]:
pathway_configs = read.csv('configurations/06_Pathway_Configs.csv', sep = ',')
pathway_configs = pathway_configs[pathway_configs$mofa_result_name != '',]

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on 'configurations/06_Pathway_Configs.csv'"


In [575]:
head(pathway_configs)

,mofa_result_name,factor_set,coverage_par,types,coverage_plot,p_value_plot,max_pathways_plot,enrichment_plot,top_features_plot,pathway_selection
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<lgl>
1,MI_v1_MOFA,"1,2,3,4,5",0.2,"B.cell,CD16.Mono,CD4.CTL,CD4.Naive,CD4.TCM,CD4.TEM,cDC2,gdT,neutrophil,NK,Treg",0.5,0.05,8,positive,0.125,NA


In [576]:
### Generate the result data directory if it does not exist yet
if(!file.exists(paste0(result_path, '06_results'))){
    dir.create(file.path(paste0(result_path, '06_results')))
    }

# Define parameters 

In [577]:
### Save values from loaded config file in variables

In [578]:
## For the calculation of the pathway enrichment

In [579]:
mofa_name = pathway_configs$mofa_result_name[1]   # mofa results that should be used

In [580]:
factor_set = as.numeric(unlist(str_split(pathway_configs$factor_set[1], ',')))  # factors for which enrichment analysis should be executed

In [581]:
factor_set

[1] 1 2 3 4 5

In [582]:
length(factor_set)

[1] 5

In [583]:
### Factor set needs to be at least first to factors if factor-set only one factor does not work

In [584]:
if(length(factor_set) <= 1){
    if(factor_set <= 1){
        factor_set = c(1,2)}
    }
if(length(factor_set) <= 1){
    if(factor_set > 1){
        factor_set = 1:factor_set
        }
    }

In [585]:
factor_set

[1] 1 2 3 4 5

In [586]:
coverage_par = pathway_configs$coverage_par[1] # coverage parameter: how many of the genes of a pathway need to be included in the MOFA feature set for testing this pathway for enrichment

In [587]:
coverage_par

[1] 0.2

In [588]:
views_set = unlist(str_split( pathway_configs$types[1], ','))  # extract the views for which a view-specific pathway enrichment analysis should be executed

In [589]:
views_set

[1] "B.cell"     "CD16.Mono"  "CD4.CTL"    "CD4.Naive"  "CD4.TCM"   
 [6] "CD4.TEM"    "cDC2"       "gdT"        "neutrophil" "NK"        
[11] "Treg"

In [590]:
## Parameters for the visualization of pathways

In [591]:
### Select pathways based on thresholds (like coverage, p-value, direction of enrichment)
coverage_plot = pathway_configs$coverage_plot[1]
p_value_cutoff_plot =pathway_configs$p_value_plot[1]
max_pathways =pathway_configs$max_pathways_plot[1]
select_enrichment = pathway_configs$enrichment_plot[1]

### Alternative: select pathways based on their specified names
pathway_selection_var =pathway_configs$pathway_selection[1]

In [592]:
pathway_selection_var

[1] NA

In [593]:
### For visualization define which genes should be ploted (need to be among the top x% of features for the Factor)
top_var_thres =pathway_configs$top_features_plot[1] # choose threshold of top x % of features of MOFA factor to take into account

In [594]:
## Fixed parameters (may be modified here)

In [595]:
## For enrichment calculation
use_statistic = "rank.sum" # which statistic to use to calcuate the enrichment; alternatives: mean.diff, rank.sum
use_test = 'parametric'  # which test to use to test the enrichment; alternatives: permutation, parametric, "cor.adj.parametric"
p_val_cutoff = 0.05
min_size = 5

# Load Data 

## Model Data

In [596]:
### Load the trained MOFA Model

In [597]:
model_name =  paste0("03_MOFA_MODEL_", mofa_name,'.hdf5')

In [598]:
outfile = file.path( paste0(result_path, '/03_results/',  model_name) )

In [599]:
outfile

[1] "/home/icb/corinna.losert/projects/mofa_workflow/result_data//03_results/03_MOFA_MODEL_MI_v1_MOFA.hdf5"

In [600]:
if(file.exists(outfile)){
    model <- load_model(outfile, verbose = TRUE)
    popup_function_pos(paste0('Loaded: ', model_name))
    } else {popup_function_neg(paste0('Error: ',result_path, '/03_results/ ', model_name, 'could not be loaded. Check whether the previous steps have been executed successfully')) } 

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
"Factor(s) 5, 9, 18 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
"
Checking for highly correlated factors...



## Pathways

In [601]:
### Load the pre-defined pathway set( needs to include the columns:
# ID (unique identifier of the pathway)
# gene : gene-symbol of the gene belonging to the pathway (will be matched to the MOFA features)
# pathway_name: a textual description of the pathway 

In [602]:
path = paste0(data_path, 'Prepared_Pathway_Data.csv')
if(file.exists(path)){
    pathways =  read.csv(path)
    pathways$X = NULL 
    popup_function_pos(paste0('Loaded: ','Prepared_Pathway_Data.csv'))
    

} else{  popup_function_neg(paste0('Error: ','Prepared_Pathway_Data.csv', ' could not be loaded. Make sure the file is included within the specified input data folder'))
       print(paste0('Error: ','Prepared_Pathway_Data.csv', ' could not be loaded. Make sure the file is included within the specified input data folder'))
      
      
      }

In [603]:
head(pathways,2)

,ID,gene,pathway_name
,<chr>,<chr>,<chr>
1,R-HSA-1059683,JAK1,Interleukin-6 signaling
2,R-HSA-1059683,STAT1,Interleukin-6 signaling


# Prepare model data

## Extract the weights from the model

In [604]:
### Get the feature weights from the model and prepare the format

In [605]:
weights = get_weights(model, views = "all", factors = "all")
weight_data = data.frame()
for (i in names(weights)){
    data = data.frame(weights[[i]])
    data$type = i
    weight_data = rbind(weight_data,data)
}
weight_data$variable_name = rownames(weight_data)
weight_data$view <- weight_data$type 
weight_data$gene = sapply(strsplit(weight_data$variable_name, "_"), "[", 3)
head(weight_data)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,...,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,type,variable_name,view,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
B.cell__ACTB,-0.18622572,0.2532893,0.16641189,-0.052115239,-0.12850895,-0.04607836,0.29499641,-0.014377703,0.0030737847,-0.23742092,...,0.08731972,0.022657087,-0.013662204,0.02398114,0.25733860,-0.33695142,B.cell,B.cell__ACTB,B.cell,ACTB
B.cell__ACTG1,-0.05929283,-0.0210286,0.12941701,-0.006415541,-0.10125865,-0.02679262,0.17690643,-0.012719177,-0.0073429115,0.05831917,...,0.07711172,0.065671454,-0.008466872,0.23901791,0.71884011,0.04404755,B.cell,B.cell__ACTG1,B.cell,ACTG1
B.cell__ACTR2,-0.03962156,0.2723610,0.07830457,-0.174129744,0.03383196,0.08418610,0.58085032,0.006520657,0.0005309017,-0.34673154,...,-0.56082421,0.050565496,-0.009194180,0.39254573,0.33316832,-0.31498583,B.cell,B.cell__ACTR2,B.cell,ACTR2
B.cell__ACTR3,0.06087441,0.0377486,-0.06697696,-0.317071164,-0.01804716,-0.06606528,0.01939307,-0.037956857,0.0048412515,0.02484377,...,-0.20704329,0.006030756,-0.111238155,0.30912416,0.17350324,0.19337951,B.cell,B.cell__ACTR3,B.cell,ACTR3
B.cell__ADD3,-0.08900485,0.1829813,0.05449551,-0.208797754,-0.03130545,-0.04623831,0.07433143,0.008030978,-0.0080186938,-0.26903606,...,0.06001593,-0.036961132,-0.281694254,0.13583734,0.12275302,0.08883714,B.cell,B.cell__ADD3,B.cell,ADD3
B.cell__AES,0.16131069,-0.1939779,0.02168047,0.164927176,-0.15197433,0.16541432,0.03628287,0.030639418,-0.0121872510,-0.25716339,...,0.03850664,0.052330404,0.037598427,0.75380586,0.05670264,-0.10799185,B.cell,B.cell__AES,B.cell,AES


In [606]:
## Transform to long format

In [607]:
feature_weights_long = melt(weight_data)

Using type, variable_name, view, gene as id variables



In [608]:
head(feature_weights_long,2)

,type,variable_name,view,gene,variable,value
,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,B.cell,B.cell__ACTB,B.cell,ACTB,Factor1,-0.18622572
2,B.cell,B.cell__ACTG1,B.cell,ACTG1,Factor1,-0.05929283


## Adjust feature names in the model

In [609]:
### Feature names should map to the genes in the pathway set (therefore the view component is  removed)

In [610]:
head(features_names(model)[[1]] )

[1] "B.cell__ACTB"  "B.cell__ACTG1" "B.cell__ACTR2" "B.cell__ACTR3"
[5] "B.cell__ADD3"  "B.cell__AES"

In [611]:
model_conc = model ## save original model for later
for(i in names(features_names(model))){   
        features_names(model)[[i]] = with(feature_weights_long, gene[match(features_names(model)[[i]], variable_name)])
    }

In [612]:
head(features_names(model)[[1]] )

[1] "ACTB"  "ACTG1" "ACTR2" "ACTR3" "ADD3"  "AES"

## Create a MOFA model for the overall enrichment analysis across views

In [613]:
# We need a model that has features of all views concatenated in a single view

In [614]:
views <- names(features_names(model_conc))
tmp <- sapply(views, function(view) model_conc@intercepts[[view]]$group1)
names(tmp) <- NULL  
model_conc@intercepts[['complete']]$group1 = unlist(tmp)
model_conc@expectations$W[['complete']] = do.call(rbind, lapply(views, function(view) model_conc@expectations$W[[view]]))
model_conc@features_metadata = rbind(model_conc@features_metadata, 
                                     data.frame(feature = model_conc@features_metadata$feature, view="complete"))
model_conc@dimensions$D['complete'] = sum(model_conc@dimensions$D)
model_conc@data[['complete']]$group1 = do.call(rbind, lapply(views, function(view) model_conc@data[[view]]$group1))
model_conc@data_options$views = c(model_conc@data_options$views , 'complete')
model_conc@model_options$likelihoods['complete'] = 'gaussian'
model_conc@dimensions$M = length(views) + 1

model_conc@cache$variance_explained$r2_total$group1['complete'] = mean(model_conc@cache$variance_explained$r2_total$group1)
model_conc@cache$variance_explained$r2_per_factor$group1 = cbind(model@cache$variance_explained$r2_per_factor$group1, data.frame('complete' = rowMeans(model_conc@cache$variance_explained$r2_per_factor$group1)))

In [615]:
model_conc

Trained MOFA with the following characteristics: 
 Number of views: 18 
 Views names: B.cell CD14.Mono CD16.Mono CD4.CTL CD4.Naive CD4.TCM CD4.TEM CD8.Naive CD8.TEM NK Treg cDC2 clinical cytokine gdT neutrophil proteomic complete 
 Number of features (per view): 686 1589 1059 542 473 661 737 397 570 508 364 280 4 66 218 981 490 9625 
 Number of groups: 1 
 Groups names: group1 
 Number of samples (per group): 128 
 Number of factors: 20 


# Prepare pathway data

In [616]:
### Prepare the pathway data to use for the enrichment

In [617]:
head(pathways,2)

,ID,gene,pathway_name
,<chr>,<chr>,<chr>
1,R-HSA-1059683,JAK1,Interleukin-6 signaling
2,R-HSA-1059683,STAT1,Interleukin-6 signaling


## Filter pathways out based on coverage

In [618]:
## get all features included in the MOFA model
mofa_genes = data.frame(gene = unique(feature_weights_long$gene), is_feature = 1)

In [619]:
head(mofa_genes,2)

,gene,is_feature
,<chr>,<dbl>
1,ACTB,1
2,ACTG1,1


In [620]:
### Merge features within mofa model to pathways
feature_set = merge(pathways, mofa_genes, all.x = TRUE)  

In [621]:
head(feature_set,2)

,gene,ID,pathway_name,is_feature
,<chr>,<chr>,<chr>,<dbl>
1,A1BG,R-HSA-168256,Immune System,1
2,A1BG,R-HSA-6798695,Neutrophil degranulation,1


In [622]:
### Remove pathways for which we have not a high amount of genes in our data (coverage_par)
filter = feature_set %>% group_by(ID, pathway_name) %>% summarise(gene_amount = n(),matched_amount = sum(!is.na(is_feature)),  coverage = sum(!is.na(is_feature)) / n()) %>% filter(coverage >=  coverage_par)

### Get the pathways that have been filtered out because of to low coverage
filtered_pathways = feature_set %>% group_by(ID, pathway_name) %>% summarise(gene_amount = n(),matched_amount = sum(!is.na(is_feature)),  coverage = sum(!is.na(is_feature)) / n()) %>% filter(coverage <  coverage_par)
nrow(filtered_pathways)  # amount of pathways that have een filtred out

`summarise()` has grouped output by 'ID'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'ID'. You can override using the `.groups` argument.


[1] 17

In [623]:
head(filter,2)

ID,pathway_name,gene_amount,matched_amount,coverage
<chr>,<chr>,<int>,<int>,<dbl>
R-HSA-1059683,Interleukin-6 signaling,15,11,0.7333333
R-HSA-1168372,Downstream signaling events of B Cell Receptor (BCR),101,39,0.3861386


In [624]:
feature_set = merge(feature_set, filter)

In [625]:
# Overview pathways that have been excluded from testing due to low amount of matching genes
head(filtered_pathways,15) 

ID,pathway_name,gene_amount,matched_amount,coverage
<chr>,<chr>,<int>,<int>,<dbl>
R-HSA-1170546,Prolactin receptor signaling,21,3,0.1428571
R-HSA-1461957,Beta defensins,40,2,0.0500000
R-HSA-1461973,Defensins,50,6,0.1200000
R-HSA-1606322,ZBP1(DAI) mediated induction of type I IFNs,24,4,0.1666667
R-HSA-1606341,IRF3 mediated activation of type 1 IFN,8,0,0.0000000
R-HSA-199920,CREB phosphorylation,7,1,0.1428571
R-HSA-3248023,Regulation by TREX1,1,0,0.0000000
R-HSA-450321,JNK (c-Jun kinases) phosphorylation and activation mediated by activated human TAK1,23,3,0.1304348
R-HSA-5669034,TNFs bind their physiological receptors,29,4,0.1379310


In [626]:
### Remove NA entries for not mapped genes in feature set

In [627]:
head(feature_set,2)

,ID,pathway_name,gene,is_feature,gene_amount,matched_amount,coverage
,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>
1,R-HSA-1059683,Interleukin-6 signaling,PTPN11,NA,15,11,0.7333333
2,R-HSA-1059683,Interleukin-6 signaling,IL6R,1,15,11,0.7333333


In [628]:
feature_set = feature_set[!is.na(feature_set$is_feature),]

In [629]:
nrow(feature_set)

[1] 5564

In [630]:
head(feature_set,2)

,ID,pathway_name,gene,is_feature,gene_amount,matched_amount,coverage
,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>
2,R-HSA-1059683,Interleukin-6 signaling,IL6R,1,15,11,0.7333333
3,R-HSA-1059683,Interleukin-6 signaling,JAK1,1,15,11,0.7333333


In [631]:
### Save coverage for later usage
coverage_info = unique(feature_set[,c('ID', 'pathway_name', 'coverage')])

In [632]:
### Get genes that are not mapped / and add a not mapped pathway to use them in the background later
# (this is for including all features that are in the MOFA model in the enrichment analysis, either as belonging to a pathway or only being part of the background set)

In [633]:
non_pathway_genes = unique(mofa_genes$gene[!mofa_genes$gene %in% unique(feature_set$gene)])

In [634]:
add_pathway = data.frame(ID = 'Background', pathway_name = 'Background', gene = non_pathway_genes, 
                         is_feature = 1, gene_amount = length(non_pathway_genes), matched_amount =  length(non_pathway_genes),
                         coverage = 1)

In [635]:
feature_set = rbind(feature_set, add_pathway)

## Transform to binary matrix format

In [636]:
### Transform the pathway feature set to a binary matrix format (1 indicating that a certain feature belongs to the pathway; 0 that it does not)

In [637]:
head(feature_set,2)

,ID,pathway_name,gene,is_feature,gene_amount,matched_amount,coverage
,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>
2,R-HSA-1059683,Interleukin-6 signaling,IL6R,1,15,11,0.7333333
3,R-HSA-1059683,Interleukin-6 signaling,JAK1,1,15,11,0.7333333


In [638]:
feature_set$pathway_id = paste0(feature_set$ID, '_', feature_set$pathway_name)

In [639]:
feature_set$value = 1

In [640]:
### Adjust names for overall approach (concatenate gene names with cell-types and match to pathways)

In [641]:
feature_weights_long_mapped_join = unique(feature_weights_long[,c('gene', 'variable_name')])

In [642]:
head(feature_weights_long_mapped_join,2)

,gene,variable_name
,<chr>,<chr>
1,ACTB,B.cell__ACTB
2,ACTG1,B.cell__ACTG1


In [643]:
feature_set_all = merge(feature_set, feature_weights_long_mapped_join, by.x = 'gene', by.y = 'gene')

In [644]:
nrow(feature_set_all)

[1] 31719

In [645]:
### Binarize feature set to use in type-specific enrichment

In [646]:
## Based on gene-name for overall enrichment

feature_set = unique(feature_set[,c('pathway_id', 'gene', 'value')]) %>% dcast(pathway_id ~ gene, value.var = 'value')
feature_set[is.na(feature_set)]= 0

In [647]:
head(feature_set,2)

,pathway_id,A1BG,A2M,AAK1,AATK,ABCA7,ABHD17A,ABHD2,ABHD5,ABI1,...,ZNF467,ZNF511.PRAP1,ZNF706,ZNFX1,ZNHIT1,ZRANB2,ZSWIM6,ZYX,hCG,sCD40L
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Background_Background,0,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2,R-HSA-1059683_Interleukin-6 signaling,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [648]:
### Binarize feature set to use in overall enrichment

In [649]:
feature_set_all = unique(feature_set_all[,c('pathway_id', 'variable_name', 'value')]) %>% dcast(pathway_id ~ variable_name, value.var = 'value')
feature_set_all[is.na(feature_set_all)] = 0

In [650]:
head(feature_set_all,2) # feature set containing only genes

,pathway_id,B.cell__ACTB,B.cell__ACTG1,B.cell__ACTR2,B.cell__ACTR3,B.cell__ADD3,B.cell__AES,B.cell__AFF3,B.cell__AHNAK,B.cell__AKAP13,...,proteomic__WWC3_Q9ULE0,proteomic__X_A0A0G2JRQ6,proteomic__X_A0A0J9YW62,proteomic__X_A0A1B0GU03,proteomic__X_A0A2R8Y7X9,proteomic__X_B4E1Z4,proteomic__X_H0YJW9,proteomic__YWHAZ_P63104,proteomic__ZNF511.PRAP1_H7BY64,proteomic__hCG_2039566_A0A0U1RR32
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Background_Background,0,0,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
2,R-HSA-1059683_Interleukin-6 signaling,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [651]:
## Adjust rownames and convert to matrix

In [652]:
rownames(feature_set) = feature_set$pathway_id
rownames(feature_set_all) = feature_set_all$pathway_id

In [653]:
feature_set$pathway_id = NULL
feature_set_all$pathway_id = NULL

In [654]:
feature_set = as.matrix(feature_set)
feature_set_all = as.matrix(feature_set_all)

# Run pathway enrichment

## Per View

In [655]:
### Filter views_set on relevant views with overlaps to pathway features

In [656]:
views_set

[1] "B.cell"     "CD16.Mono"  "CD4.CTL"    "CD4.Naive"  "CD4.TCM"   
 [6] "CD4.TEM"    "cDC2"       "gdT"        "neutrophil" "NK"        
[11] "Treg"

In [657]:
views_set = views_set[views_set %in% unique(str_replace(colnames(feature_set_all)[colSums(feature_set_all[rownames(feature_set_all) != 'Background_Background',]) > 5], '__.*', ''))]

In [658]:
views_set

[1] "B.cell"     "CD16.Mono"  "CD4.CTL"    "CD4.Naive"  "CD4.TCM"   
 [6] "CD4.TEM"    "cDC2"       "gdT"        "neutrophil" "NK"        
[11] "Treg"

In [659]:
## For each view seperately run the enrichment

In [660]:
if(max(views_set) != ''){
    enrichment_result_types = run_enrichment_pathway(
        model = model, # MOFA Model
        factor_set = factor_set, # list of factors for which to run the enrichment
        views = views_set,
        use_statistic = use_statistic, # which statistic to use
        feature_set = feature_set, # Pathway Feature Set mapping; here use without type names
        min_size = min_size, # Min size of genes within a pathway
        use_test = use_test, # test used for calculating p-value
        p_val_cutoff = p_val_cutoff, # p-value cutoff used
        enrichment_result_p_val = data.frame())# dataset for saving result 
    } else{enrichment_result_types = ''}

Intersecting features names in the model and the gene set annotation results in a total of 686 features.


Running feature set Enrichment Analysis with the following options...
View: B.cell 
Number of feature sets: 73 
Set statistic: rank.sum 
Statistical test: parametric 





Using pathway as id variables

Using pathway as id variables

Intersecting features names in the model and the gene set annotation results in a total of 686 features.


Running feature set Enrichment Analysis with the following options...
View: B.cell 
Number of feature sets: 73 
Set statistic: rank.sum 
Statistical test: parametric 


Subsetting weights with negative sign




Using pathway as id variables

Using pathway as id variables

Intersecting features names in the model and the gene set annotation results in a total of 686 features.


Running feature set Enrichment Analysis with the following options...
View: B.cell 
Number of feature sets: 73 
Set statistic: rank.sum 
Statistical test: parametric 


Sub

In [661]:
# Example of the resulting enrichment table

In [662]:
if(is.data.frame(enrichment_result_types)){
    head(enrichment_result_types  %>% arrange(padj) ,5)}

,pathway,variable,p,padj,view,enrichment,global_FDR
,<chr>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,R-HSA-168249_Innate Immune System,Factor4,7.793962e-05,0.001851066,CD16.Mono,negative,0.2113847
2,R-HSA-168256_Immune System,Factor4,5.173760e-05,0.001851066,CD16.Mono,negative,0.2113847
3,R-HSA-6798695_Neutrophil degranulation,Factor4,6.436162e-05,0.001851066,CD16.Mono,negative,0.2113847
4,Background_Background,Factor2,8.709710e-05,0.002645071,CD4.TEM,all,0.2113847
5,R-HSA-166016_Toll Like Receptor 4 (TLR4) Cascade,Factor2,1.522842e-04,0.002645071,CD4.TEM,all,0.2568186


## Across Views

In [663]:
## Run an enrichment analysis across all the views

In [664]:
enrichment_result_all = run_enrichment_pathway(
    model = model_conc, # MOFA Model
    views = 'complete', # dimensions for which to run
    factor_set = factor_set, # list of factors for which to run the enrichment
    use_statistic = use_statistic, # which statistic to use
    feature_set = feature_set_all, # Pathway Feature Set mapping; use the concatenated one with cell-types
    min_size = min_size, # Min size of genes within a pathway
    use_test = use_test, # test used for calculating p-value
    p_val_cutoff = p_val_cutoff, # p-value cutoff used
    enrichment_result_p_val = data.frame()) # dataset for saving results

Warning message in run_enrichment(model, view = i, factors = factor_set, set.statistic = c(use_statistic), :
"7 features were removed because they had no variance in the data.
"
Intersecting features names in the model and the gene set annotation results in a total of 9618 features.


Running feature set Enrichment Analysis with the following options...
View: complete 
Number of feature sets: 165 
Set statistic: rank.sum 
Statistical test: parametric 





Using pathway as id variables

Using pathway as id variables

Warning message in run_enrichment(model, view = i, factors = factor_set, set.statistic = c(use_statistic), :
"7 features were removed because they had no variance in the data.
"
Intersecting features names in the model and the gene set annotation results in a total of 9618 features.


Running feature set Enrichment Analysis with the following options...
View: complete 
Number of feature sets: 165 
Set statistic: rank.sum 
Statistical test: parametric 


Subsetting weights 

In [665]:
head(enrichment_result_all %>% arrange(padj) ,5)

,pathway,variable,p,padj,view,enrichment,global_FDR
,<chr>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,R-HSA-168928_DDX58/IFIH1-mediated induction of interferon-alpha/beta,Factor1,4.925678e-19,8.127369e-17,complete,negative,1.219105e-15
2,R-HSA-1236978_Cross-presentation of soluble exogenous antigens (endosomes),Factor1,1.084224e-16,1.788969e-14,complete,positive,1.341727e-13
3,R-HSA-1169408_ISG15 antiviral mechanism,Factor1,1.065902e-15,8.054560e-14,complete,negative,8.793692e-13
4,R-HSA-1169410_Antiviral mechanism by IFN-stimulated genes,Factor1,1.464466e-15,8.054560e-14,complete,negative,8.841308e-13
5,R-HSA-1222556_ROS and RNS production in phagocytes,Factor1,9.173615e-15,1.518779e-13,complete,positive,3.243528e-12


In [666]:
### Combine both versions and save the result

In [667]:
if(is.data.frame(enrichment_result_types)){
    enrichment_result = rbind(enrichment_result_all, enrichment_result_types  )
    } else{enrichment_result = enrichment_result_all}

In [668]:
enrichment_result$ID = str_replace(enrichment_result$pathway, '_.*', '')
enrichment_result$pathway_name = str_replace(enrichment_result$pathway, '.*_', '')

In [669]:
enrichment_result$pathway = NULL

In [670]:
enrichment_result = merge(enrichment_result, coverage_info)

In [671]:
write.csv(enrichment_result, paste0(result_path, '/06_results/06_Pathway_Enrichment_' ,mofa_name, '.csv'), row.names = FALSE)

# Plot interesting pathways

In [672]:
## Plot a subset of the enriched pathways (based on the parameters of the config file)

In [673]:
pathway_selection_var  # specifies whether a specific pathway has been selected in the config file

[1] NA

In [674]:
# specifies whether pathways will be displayed based on p-value and coverage thresholds specified in the config file
# or specifically selected pathways will be shown
if(pathway_selection_var == '' | is.na(pathway_selection_var) ){
    pathways_selection = enrichment_result[
        (enrichment_result$coverage > coverage_plot) & 
        (enrichment_result$enrichment == select_enrichment) &
        (enrichment_result$global_FDR < p_value_cutoff_plot),] %>%  arrange(padj) %>% group_by(variable) %>% top_n(max_pathways, -padj)
    } else{ pathways_selection = enrichment_result[enrichment_result$pathway_name %in% unlist(str_split(pathway_selection_var, ',')),]}

In [675]:
## Select the genes of the pathway that have the highest weights on the selected factor

In [676]:
### Get involved genes

## Define gene-set to merge
geneset_oi_pos_per_factor = feature_weights_long %>% group_by(variable) %>% arrange( desc(value),  .by_group = TRUE)  %>% top_frac(as.numeric(top_var_thres))
geneset_oi_pos_per_factor$direction = 'positive'
geneset_oi_neg_per_factor = feature_weights_long %>% group_by(variable) %>% arrange(desc(value),  .by_group = TRUE)  %>% top_frac(-as.numeric(top_var_thres))
geneset_oi_neg_per_factor$direction = 'negative'
geneset_oi = rbind(geneset_oi_pos_per_factor, geneset_oi_neg_per_factor)

colnames(geneset_oi) = c('type', 'variable_name','view',  'gene', 'variable', 'factor_value', 'factor_value_direction')
head(geneset_oi,2)

Selecting by value
Selecting by value


type,variable_name,view,gene,variable,factor_value,factor_value_direction
<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>,<chr>
CD4.TCM,CD4.TCM__GMFG,CD4.TCM,GMFG,Factor1,0.5732698,positive
CD4.TCM,CD4.TCM__EDF1,CD4.TCM,EDF1,Factor1,0.5378782,positive


In [677]:
### Merge genes belongig to pathway to enriched pathway sets

pathways_vis_genes = merge(pathways_selection, pathways) %>% mutate(pvalue=p, view_text=view, cluster=view, name=pathway_name) %>% dplyr::select(-view)

In [678]:
### Add the feature weights to the corresponding genes from geneset_oi

In [679]:
pathways_vis_genes = merge(pathways_vis_genes, geneset_oi, by.x = c('gene', 'variable'), by.y = c('gene','variable' ))

In [680]:
### Summarise to get max/ mean factor value of each gene per pathway (remove cell-cluster/view/type dimension)  --> tBD max or mean (for max --> absolute value?)

pathways_vis_genes_summarized = pathways_vis_genes %>% group_by(gene ,variable,  pathway_name, ID, name, cluster ,view) %>% summarise(factor_value = mean(factor_value), enrichment_type = paste0(unique(enrichment), collapse = '&'), pvalue = min(pvalue))

`summarise()` has grouped output by 'gene', 'variable', 'pathway_name', 'ID', 'name', 'cluster'. You can override
using the `.groups` argument.


In [681]:
head(pathways_vis_genes_summarized,2)

gene,variable,pathway_name,ID,name,cluster,view,factor_value,enrichment_type,pvalue
<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
C3,Factor3,Alternative complement activation,R-HSA-173736,Alternative complement activation,complete,proteomic,-0.4621188,positive,0.002495833
C5AR1,Factor4,Complement cascade,R-HSA-166658,Complement cascade,complete,CD14.Mono,0.2539385,positive,0.001239171


In [682]:
### Plot the pathways with summarized gene values

In [683]:
pathways_sum_plot = list()

In [684]:
for(i in unique(pathways_vis_genes_summarized$variable)){

    xlabel = xlab('Gene') 
    ylabel = ylab('Pathway')
    
    pathways_sum_plot[[i]] = ggplot(pathways_vis_genes_summarized[pathways_vis_genes_summarized$variable == i,], aes(gene,  pathway_name, fill= factor_value)) + 
        plot_config_heatmap + 
        geom_tile() + 
        scale_fill_gradient2(low = "#1D2ED8", mid = "white", high = "#D8911D", midpoint = 0)  + 
        scale_x_discrete(position = "top") +
        theme(axis.text.x = element_text(angle = 90), axis.title.x = element_blank(), axis.text.y = element_text(hjust = 0, vjust = 0.5)) +
        xlabel +
        ylabel +  scale_y_discrete(labels = label_wrap(50))+
        ggtitle(paste0(i, ' values of pathway genes (top ', top_var_thres  * 2 *100, '% of features)'))
    }

In [685]:
options(repr.plot.width=20, repr.plot.height=10)
#pathways_sum_plot[[i]] +  scale_y_discrete(labels = label_wrap(25))

In [686]:
## Plot factor values for all genes

In [687]:
pathways_detail_plot = list()

In [688]:
### Visualized the exact factor values of the genes
for(i in unique(pathways_vis_genes$variable)){
    # Specific Text Descriptions:
    xlabel = xlab('Gene') 
    ylabel = ylab('View')

    plot_data_cluster = unique(pathways_vis_genes[pathways_vis_genes$variable == i,c('gene', 'variable', 'type','view',  'variable_name', 'factor_value', 'factor_value_direction')])

    pathways_detail_plot[[i]] = ggplot(plot_data_cluster, aes(gene,  view, fill= factor_value)) + 
        plot_config_heatmap + 
        geom_tile() + 
        scale_fill_gradient2(low = "#1D2ED8", mid = "white", high ="#D8911D", midpoint = 0)  + 
        scale_x_discrete(position = "top") +
        theme(axis.text.x = element_text(angle = 90)) +
        xlabel +
        ylabel
}

In [689]:
#pathways_detail_plot[[i]]

In [690]:
### Combine plot and save

In [691]:
combined_plot = list()

In [692]:
for(i in 1:length(pathways_detail_plot)){
    combined_plot[[i]] = ggarrange(pathways_sum_plot[[i]],
              pathways_detail_plot[[i]] + theme(axis.text.x = element_blank(), axis.title.x =element_blank()), align = 'v', ncol = 1)
}

In [693]:
#combined_plot[[i]]

In [694]:
## Save the plot

In [695]:
figure_name = paste0( "FIG06_Pathways_and_Genes")

In [696]:
# Sizes of the plot
width_par = 8.07
height_par = 4

In [697]:
pdf(paste0('figures/06_figures/', figure_name, '_',   mofa_name, '.pdf'), width =width_par, height =height_par)

for(j in 1:length(combined_plot)){
        print( combined_plot[[j]])
        }
dev.off()   


png 
  2

In [698]:
### Inform about execution finalization
popup_function_pos('06_Downstream_Pathways: Execution Finished')

In [699]:
Sys.sleep(20)
popup_function_info('06_Downstream_Pathways')